In [1]:
from source.regency import Regency
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')


seasons = 10
Game = Regency()
clear_output()

for a in range(seasons):
    print(a)
    Game.random_events()
    Game.collect_regency_points()
    Game.domain_initiative()
    Game.collect_gold_bars()
    Game.maintenance_costs(Update=True)
    Game.take_domain_actions()
    Game.war_move()
Game.clear_screen()
import pandas as pd
df = pd.concat([Game.Seasons[b]['Actions'][a+1] for a in range(3) for b in range(seasons)])
print(set(df['Action']))
Game.War

0
Instructions for updating:
Use tf.cast instead.


KeyError: 'Level'

In [ ]:
Game.show_map(caravans=True, regions=['Eastern Marches'], domains=['Elinie'])

In [ ]:
Game.Projects

In [ ]:
df[df['Action'].str.contains('build')]

In [ ]:
Game.errors

In [ ]:
Game.Projects